In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported

from trl import SFTTrainer
from transformers import TrainingArguments
from tuning.data.train_dataset import get_train_dataset
from tuning.training.config_training import ModelLoadConfig, LoraConfig, SFTRunConfig, PTRunConfig, DPOTrainingConfig, TrainingArgumentsConfig, PassAtKConfig, sft_batch_size, effective_batch_size
from tuning.training.perplexity_callback import PerplexityStoppingCallback
from tuning.training.passk_callback import PassAtKStoppingCallback
from tuning.utils.utils import apply_chat_template, chat_template_func
import json
import sys
from datasets import load_from_disk
from typing import List, Optional, Union
from pathlib import Path
from tuning.config import DATASETS_DIR, HF_MODEL_MAP
import os
from tuning.training.config_training import DatasetConfig, SFTRunConfig
from tuning.config import MODELS_DIR
from tuning.training.sft_training import train_model_sft
from tuning.training.dpo_training import train_model_dpo
import torch

import subprocess
import importlib

In [23]:
import importlib
import tuning.training.passk_callback
importlib.reload(tuning.training.passk_callback)
from tuning.training.passk_callback import PassAtKStoppingCallback
from tuning.training.dpo_training import train_model_dpo
importlib.reload(tuning.training.dpo_training)
from tuning.training.dpo_training import train_model_dpo

In [2]:
MODEL = "llama3-8B"
total_train_size = 8192  # 29980
perplexity_thresholds = [7.0,6.0, 5.75, 5.5, 5.25, 5.0, 4.75, 4.5, 4.25,4.0, 3.9, 3.8, 3.7, 3.6,3.55,3.5,3.45,3.4,3.35,3.3, 3.25, 3.2, 3.15, 3.1]
perplexity_thresholds = [7.0, 6.0]

In [10]:
dataset_config = DatasetConfig(
    dataset = "tuluif",
    dataset_type = "sft",
    train_size = total_train_size, # 29980
)

run_config = SFTRunConfig(
    dataset_config = dataset_config,
    model_name_hf = HF_MODEL_MAP[MODEL],  # Use HuggingFace model name, not local path
    model_name = MODEL,  # Base model name for output directory construction
    do_training=True,
    do_inference=False,
    do_evaluation=False,
)
passk_config = PassAtKConfig( # this is just to dynamically view the pass@1 of ifeval
    target_pass_at_k=[0.3,0.4,0.5,0.6],
    k_values=[1],
    n_samples=1,
    num_prompts=100,
    temperature=0.7,
    strict=True,
    enabled=True,
)

lora_config = LoraConfig()
model_load_config = ModelLoadConfig()
model_load_config.max_seq_length = 4096
training_args = TrainingArgumentsConfig()

In [4]:
model, tokenizer, trainer, callbacks = train_model_sft(
    run_config = run_config,
    lora_config = lora_config,
    model_load_config = model_load_config,
    training_args = training_args,
    perplexity_thresholds = perplexity_thresholds, 
)

Getting train dataset for run config: llama3-8B_sft-tuluif-8192
Checking for dataset at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/sft-tuluif-8192
Dataset already exists at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/sft-tuluif-8192
Sampled dataset: DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints'],
        num_rows: 8192
    })
    test: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints'],
        num_rows: 200
    })
})
Example training row: {'id': 'personas_IF_9u3mcrurksv7hypq3xlppyba', 'prompt': 'Compile a detailed summary of the most recent case discussed on the podcast. Include the title of the episode wrapped in double angular brackets, i.e. <<title>>, and quote at least one statement made by the host during the discussion.', 'messages': [{'content': 'You are a helpful assistant who is an expert at responding to prompts by carefully followin

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.


DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints', 'text'],
        num_rows: 8192
    })
    test: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints', 'text'],
        num_rows: 200
    })
})
{'id': 'personas_IF_9u3mcrurksv7hypq3xlppyba', 'prompt': 'Compile a detailed summary of the most recent case discussed on the podcast. Include the title of the episode wrapped in double angular brackets, i.e. <<title>>, and quote at least one statement made by the host during the discussion.', 'messages': [{'content': 'You are a helpful assistant who is an expert at responding to prompts by carefully following the given instructions', 'role': 'system'}, {'content': 'Compile a detailed summary of the most recent case discussed on the podcast. Include the title of the episode wrapped in double angular brackets, i.e. <<title>>, and quote at least one statement made by the host during the discussion.', 'role': 'user'}, {'content': "I'm

Unsloth 2025.10.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


...

Checking cache directory for required files...


Unsloth: Copying 4 files from cache to `/home/shougan/projects/aip-fredashi/shougan/bala


Successfully copied all 4 files from cache to `/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-8B_sft-tuluif-8192`


Unsloth: Merging weights into 16bit: 100%|████████████████| 4/4 [01:07<00:00, 16.92s/it]


Unsloth: Merge process complete. Saved to `/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-8B_sft-tuluif-8192`


In [ ]:
ppl_callback = callbacks[-1]
metadata_file = ppl_callback.metadata_path
checkpoints = []
with open(metadata_file, "r") as f:
    for line in f:
        checkpoints.append(json.loads(line))
print(checkpoints)


[{'threshold_type': 'perplexity', 'threshold_value': 6.0, 'global_step': 40, 'checkpoint_path': '/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-8B_ppl-6.00_sft-640', 'data_points_seen': 640}]


In [ ]:
del model, tokenizer, trainer
torch.cuda.empty_cache()

In [ ]:
for checkpoint in checkpoints:
    model_name = Path(checkpoint["checkpoint_path"]).name
    data = total_train_size - checkpoint["data_points_seen"] 
    model_load_config = ModelLoadConfig()
    training_args = DPOTrainingConfig()
    training_args.eval_steps = 5
    dataset_config = DatasetConfig(
        dataset = "tuluif",
        dataset_type = "pt",
        train_size = data,
    )
    sft_run_config = SFTRunConfig(
        dataset_config = DatasetConfig(
            dataset = "tuluif",
            dataset_type = "sft",
            train_size = checkpoint["data_points_seen"],
            dynamic_path = model_name
        ),
        model_name = MODEL,
        model_name_hf = HF_MODEL_MAP[MODEL], 
        task_name = "ifeval"
    )
    run_config = PTRunConfig(
        dataset_config = dataset_config,
        # model_name_hf = HF_MODEL_MAP[MODEL],  
        model_name = MODEL,  
        sft_run_config = sft_run_config,
        task_name = "ifeval",
        pft_method = "dpo",
        do_training = True
    )
    passk_config = PassAtKConfig( # this is just to dynamically view the pass@1 of ifeval
        target_pass_at_k=[1.2],
        k_values=[1],
        n_samples=1,
        num_prompts=50,
        temperature=0.7,
        strict=True,
        enabled=True,
    )
    model, tokenizer, trainer = train_model_dpo(
        run_config = run_config,
        lora_config = lora_config,
        model_load_config = model_load_config,
        training_args = training_args,
        passk_config = passk_config,
        # perplexity_thresholds= [0.1] # dummy value to periodically check perplexities too
    )
    del model, tokenizer, trainer
    torch.cuda.empty_cache()





Per device train batch size: 1
Getting train dataset for run config: llama3-8B_llama3-8B_ppl-6.00_sft-640_pt-tuluif-7552
Checking for dataset at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/pt-tuluif-7552
Dataset already exists at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/pt-tuluif-7552
Sampled dataset: DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'constraints', 'chosen', 'rejected', 'chonsen_model', 'rejected_model', 'system_message'],
        num_rows: 7552
    })
    test: Dataset({
        features: ['id', 'prompt', 'constraints', 'chosen', 'rejected', 'chonsen_model', 'rejected_model', 'system_message'],
        num_rows: 200
    })
})
Example training row: {'id': 'personas_IF_dqxglsux2n8jeu59qktlnesh', 'prompt': 'Name two famous equestrian events that are part of the international jumping circuit, and format your answer by choosing one from these options: lowercase, UPPERCASE, Title 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

{'chonsen_model': 'gpt-4o',
 'chosen': '<|im_start|>assistant\n'
           'LONGINES GLOBAL CHAMPIONS TOUR, ROLEX GRAND SLAM OF SHOW '
           'JUMPING<|im_end|>\n',
 'constraints': ['punctuation:use no comma', 'format:choose one from options'],
 'id': 'personas_IF_dqxglsux2n8jeu59qktlnesh',
 'prompt': '<|im_start|>system\n'
           'You are a helpful assistant who is an expert at responding to '
           'prompts by carefully following the given instructions<|im_end|>\n'
           '<|im_start|>user\n'
           'Name two famous equestrian events that are part of the '
           'international jumping circuit, and format your answer by choosing '
           'one from these options: lowercase, UPPERCASE, Title '
           'Case.<|im_end|>\n'
           '<|im_start|>assistant\n',
 'rejected': '<|im_start|>assistant\n'
             '- longines global champions tour\n'
             '- FEI WORLD CUP JUMPING<|im_end|>\n',
 'rejected_model': 'gpt-4o',
 'system_message': 'You are 

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,552 | Num Epochs = 2 | Total steps = 944
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 16 x 1) = 16
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


[PassAtKCallback] on_train_begin: model_name=PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
      

...

Copied model-00004-of-00004.safetensors from local model directory


Unsloth: Merging weights into 16bit: 100%|███████████████████████████████████| 4/4 [01:17<00:00, 19.48s/it]


Unsloth: Merge process complete. Saved to `/tmp/tmp2weal79z`
[PassAtKCallback] Moving training model to CPU...
[PassAtKCallback] Loading model with vLLM from /tmp/tmp2weal79z...
LLM UTILISATION IS 0.8
INFO 01-26 03:10:13 [config.py:841] This model supports multiple tasks: {'classify', 'reward', 'generate', 'embed'}. Defaulting to 'generate'.
INFO 01-26 03:10:13 [config.py:1472] Using max model len 131072
INFO 01-26 03:10:13 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 01-26 03:10:33 [__init__.py:244] Automatically detected platform cuda.
INFO 01-26 03:10:37 [core.py:526] Waiting for init message from front-end.
INFO 01-26 03:10:37 [core.py:69] Initializing a V1 LLM engine (v0.9.2) with config: model='/tmp/tmp2weal79z', speculative_config=None, tokenizer='/tmp/tmp2weal79z', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=13107

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:09<00:27,  9.30s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:11<00:10,  5.26s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:22<00:07,  7.60s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:35<00:00,  9.88s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:35<00:00,  8.88s/it]



INFO 01-26 03:11:15 [default_loader.py:272] Loading weights took 35.66 seconds
INFO 01-26 03:11:15 [gpu_model_runner.py:1801] Model loading took 15.0006 GiB and 36.097880 seconds
INFO 01-26 03:11:28 [backends.py:508] Using cache directory: /home/shougan/.cache/vllm/torch_compile_cache/e98dde76e6/rank_0_0/backbone for vLLM's torch.compile
INFO 01-26 03:11:28 [backends.py:519] Dynamo bytecode transform time: 12.63 s
INFO 01-26 03:11:32 [backends.py:181] Cache the graph of shape None for later use
